# Cleanse Anomalies in Saildrone Salinity Data

In [1]:
def clean_anomalies(fileURL, filterMin=-1, filterMax=-1):
    """
    This function will clean a saildrone dataset of invalid/unreported data points, and optionally 
    filter the SSS data between two values (<filterMin>, <filterMax>)
    Author: Austin
    Args:
      <str>: the URL to a local saildrone dataset.
      <float>: The minimum cutoff point for all SSS data
      <float>: The maximum cutoff point for all SSS data
    Returns:
      <xr.DataSet> A clean saildrone dataset, free of invalud/unreported SSS data points.
    """
    fn = fileURL;

    #import necessary packages
    import numpy as np
    import pandas as pd
    import xarray as xr
    import matplotlib.pyplot as plt 
    import matplotlib.dates as mdates
    import seaborn as sns
    import datetime
    import warnings
    warnings.simplefilter('ignore') 

    #load map packages
    from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
    import cartopy.feature as cfeature
    import cartopy.crs as ccrs
    from calendar import month_abbr

    #open .nc file with Saildrone Data
    sail = xr.open_dataset(fn) 
    sail.close()

    #copy data for calculations versus graphing
    sail_raw = sail.copy()

    sail

    def filterSAL(min, max):
        if (max == -1):
            return False
        sail['SAL_CTD_MEAN']=sail.SAL_CTD_MEAN.where(sail.SAL_CTD_MEAN>min, np.nan)
        sail['SAL_CTD_MEAN']=sail.SAL_CTD_MEAN.where(sail.SAL_CTD_MEAN<max, np.nan)
        return True

    font = {
        'family': 'monospace',
        'color':  'k',
        'weight': 'normal',
        'size': 16,
    }

    # Remove Any Error Data
    filterSAL(0, 9999)

    min_thresh = 30
    max_thresh = 35

    if (min_thresh & max_thresh):
        print("Filtered between " + str(min_thresh) + " and " + str(max_thresh) + " PPU")
        filterSAL(min_thresh, max_thresh)

    return sail

# example call
# cleanAnomalies("~/shared/users/mgarciareyes/saildrone_data/saildrone_west_coast_survey_2019_sd1044_JPLv5.0_8dy_20210613norep_20210613_20211116.nc")